In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from collections import OrderedDict
from sklearn.preprocessing import StandardScaler
from io import StringIO
from joblib import dump, load

# Tells notebook to display all rows or columns instead of truncating
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input/jvk-ms-malware-fit"))

# Any results you write to the current directory are saved as output.

In [ ]:
clf = load('../input/jvk-ms-malware-fit/clf.joblib')
numer_columns = load('../input/jvk-ms-malware-fit/numer_columns.joblib')
cat_columns = load('../input/jvk-ms-malware-fit/cat_columns.joblib')
interaction_triples = load('../input/jvk-ms-malware-fit/interaction_triples.joblib')

In [ ]:
# TEST
#numer_columns = ['B']
#cat_columns = OrderedDict([('A0', ['1', '4', '7']), ('A1', ['2', '5', '8']), ('A2', ['3', '6', '9'])])
#interaction_triples = [[3, 3, -1.0],
# [6, 3, -1.0],
# [6, 6, -1.0],
# [9, 3, -1.0],
# [9, 6, -1.0],
# [9, 9, -1.0]]

In [ ]:
print(cat_columns)

In [ ]:
def getnth(x, n):
    try:
        return x.split('.')[n]
    except Exception:
        print("Warning: {} cannot be split; part #{} will be NULL.".format(x, n))
        return "NULL"

# Data comes in like main columns, e.g. EngineVersion is one column
# but cat_columns comes in with EngineVersion0, EngineVersion1, etc.
# proposed solution: fix the problem already with the df we read in at 
# the beginning.  This seems like the best plan, and is closest to 
# what we already do.

def splitcol(df, col):
    lst = list()
    numcols = len(df[col].iloc[0].split('.'))
    for n in range(0, numcols):
        lst.append(df[col].apply(lambda x: getnth(x, n)).astype('object').rename(col + str(n)))
    return(pd.concat(lst, axis = 1))

In [ ]:
# Takes a DataFrame along with lists of numerical and categorical columns
# The categorical columns are specified as a dictionary like: {'colname':[val1, val2, val3]}
# The returned DataFrame stores dummies under the column name 'colname_val1', etc.
def make_dummies_from_df(df, numer_columns, cat_columns, stdscale):

    if stdscale:
        df_to_return = df[numer_columns]
    else:
        df_to_return = df[numer_columns].fillna(-1)
    
    for col, valuelist in cat_columns.items():
        try:
            df_to_return = pd.concat([df_to_return, \
                                  pd.get_dummies(df[col].astype(pd.api.types.CategoricalDtype(categories = valuelist, ordered=True)), prefix = col)], \
                                  sort=False, axis=1)    
        except ValueError:
            print("ValueError in make_dummies_from_df: ", col, valuelist)
    if stdscale:
        scaler = StandardScaler()
        return np.nan_to_num(scaler.fit_transform(df_to_return.astype(float)))
    else:
        return df_to_return

# predict_write(): makes a prediction and writes it to a file
# s_buf: contains the information to be read into a df and then used to predict
# clf: classifier, which must support predict_proba()
# numer_columns: names of numerical columns
# cat_columns: names of categorical columns
# outfilename: name of output file
# mode: mode to open output file (should be append or write)
# isfirst: True if this is the first time writing to this file (so should write header)
def predict_write(s_buf, clf, numer_columns, cat_columns, outfilename, mode, isfirst, stdscale, interaction_triples, **kwargs):
    s_buf.seek(0)
    df = pd.read_csv(s_buf, **kwargs)
    # TEST df = pd.DataFrame({'A': ['1.2.3', '4.5.6', '7.8.9'], 'B': [4, 5, 6], 'HasDetections': [1, 1, 0]})
    
    expand_cols = ('EngineVersion', 'AppVersion', 'AvSigVersion', 'OsVer', 'OsBuildLab', 'Census_OSVersion')
    # TEST expand_cols = ('A')
    
    print("df.shape before expanding ", df.shape)
    
    for col in expand_cols:
        df = pd.concat((df, splitcol(df, col)), axis = 1)
        df.drop(col, inplace=True, axis = 1)
    
    # When we read in a segment of the file into s_buf, the dtypes are deduced from
    # that portion of the file only.  As a result, we cannot rely on the
    # dtypes of that segment to predict the global dtypes of that column.
    # Instead, we must use the previously determined cols_to_keep list.
    print("df.shape after expanding ", df.shape)
    df_num = make_dummies_from_df(df, numer_columns, cat_columns, stdscale)
    print("df_num.shape after dummies ", df_num.shape)
    for x in interaction_triples:
        df_num = np.concatenate((df_num, np.reshape(df_num[:, x[0]] * df_num[:, x[1]], (-1, 1))), axis = 1)
    print("df_num.shape after interactions ", df_num.shape)
    print("len(interaction_triples) ", len(interaction_triples))
    print("df_num: ", df_num)
    
    #print(df_num.shape)
    #print(df_num.iloc[0:])
    #del df
    #del df_num
    #return
    
    #TEST return
    
    with open(outfilename, mode) as outfilehandle:
        dfout = pd.DataFrame({'MachineIdentifier': df.MachineIdentifier.values, 'HasDetections': clf.predict_proba(df_num)[:,1]})
        dfout.to_csv(outfilehandle, index=False, header=isfirst)
    del df
    del df_num

# decide_test_file(): Read in infile line by line into buffer; at each progress report,
#      call predict_write to make prediction and write it to outfile.
# infile: csv file with test data
# outfile: output csv file with predictions, for judging
# clf: classifier, which must support a predict_proba method
# numer_columns: numerical columns to use for prediction
# cat_columns: categorical columns to use for prediction: a dictionary of the form
#               {'col': [val1, val2, val3], 'col2': [ ... ]}
# num_reports: number of times to print a progress report
# numrows: number of rows to read (not counting header)
# Setting numrows = N will read N + 1 rows if you count the header.
def decide_test_file(infile, outfile, clf, numer_columns, cat_columns, num_reports = 10, numrows = 80000, stdscale = False, interaction_triples = interaction_triples, **kwargs):

    rowcount = -1
    rowcount_div = 0
    start = True
    s_buf = StringIO()
    headerline = ""
    isfirst = True
    mode = 'w'
    outfilename = outfile
    infilename = infile
    
    with open(infilename, "r") as infilehandle:
        for line in infilehandle:
            if rowcount == -1: # read header row
                headerline = line
                s_buf.write(headerline)
                rowcount += 1
                continue
            elif rowcount >= numrows:
                break
            s_buf.write(line)
            rowcount += 1
            if rowcount_div >= numrows // num_reports - 1: # time for report
                os.system("echo At row: {}".format(rowcount))
                print("At row: {}".format(rowcount))
                predict_write(s_buf, clf, numer_columns, cat_columns, outfilename, mode, isfirst, stdscale, interaction_triples, **kwargs)

                isfirst = False
                mode='a'
                rowcount_div = 0
                s_buf = StringIO()
                s_buf.write(headerline) # re-initialize s_buf to just the header line
            else:
                rowcount_div += 1
        if rowcount_div > 0:
            os.system("echo Final short sections.  At row: {}". format(rowcount))
            print("Final short section.  At row: {}".format(rowcount))
            predict_write(s_buf, clf, numer_columns, cat_columns, outfilename, mode, isfirst, stdscale, interaction_triples, **kwargs)
    os.system("echo Done!  Rowcount is: {}".format(rowcount))
    print("Done!  Rowcount is: {}".format(rowcount))

In [ ]:
os.system('echo decide_test_file')
decide_test_file("../input/microsoft-malware-prediction/test.csv", "out.csv", clf, numer_columns, cat_columns, num_reports = 100, numrows=20000000, stdscale=True, interaction_triples=interaction_triples)

In [ ]:
#import itertools
#
#with open("out.csv", "r") as file:
#    for line in itertools.islice(file, 0, 10):
#        print(line, end='')

In [ ]:
#len = 0
#with open("out.csv", "r") as file:
#    for line in file:
#        len += 1
#print(len)